<a href="https://colab.research.google.com/github/kim1987/aiffel/blob/main/aiffel/deeper/dp_onefour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

deeper_14


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/colabdata/aiffel/dp_onefour/kowiki.txt.zip /content

In [ ]:
!unzip /content/kowiki.txt.zip -d /content/

Archive:  /content/kowiki.txt.zip
replace /content/kowiki.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import sentencepiece as spm
import os
import shutil

In [ ]:
from transformers import BertTokenizer, BertModel,BertConfig

In [ ]:
from transformers import XLNetTokenizer
import transformers

In [ ]:
from tqdm.notebook import tqdm

데이터 전처리

In [ ]:
my_path = "/content/kowiki.txt"
os_path = '/content/'
drive_path = '/content/drive/MyDrive/colabdata/aiffel/dp_onefour/'

In [ ]:
prefix = 'ko_32000'
vocab_size = 32000
max_len = 60

In [ ]:
if(not os.path.exists(os_path+prefix+'.model')) and os.path.exists(drive_path+prefix+'.model'):
  shutil.copyfile(drive_path+prefix+'.model',os_path+prefix+'.model')
  shutil.copyfile(drive_path+prefix+'.vocab',os_path+prefix+'.vocab')

In [ ]:
if not os.path.exists(os_path+prefix+'.model'):
  spm.SentencePieceTrainer.train(
      f"--input={my_path} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
      " --model_type=bpe" +
      " --max_sentence_length=999999" +
      " --pad_id=0 --pad_piece=[PAD]" +
      " --unk_id=1 --unk_piece=[UNK]" +
      " --bos_id=2 --bos_piece=[BOS]" +
      " --eos_id=3 --eos_piece=[EOS]" +
      " --user_defined_symbols=[SEP],[CLS],[MASK]")

In [ ]:
if not os.path.exists(drive_path+prefix+'.model'):
  shutil.copyfile(os_path+prefix+'.model',drive_path+prefix+'.model')
  shutil.copyfile(os_path+prefix+'.vocab',drive_path+prefix+'.vocab')

In [ ]:
my_tokenizer = XLNetTokenizer(os_path+prefix+'.model',
                              do_lower_case = False,
                              remove_space = True,
                              bos_token = '[BOS]',
                              eos_token = '[EOS]',
                              unk_token = '[UNK]',
                              sep_token = '[SEP]',
                              pad_token = '[PAD]',
                              cls_token = '[CLS]',
                              mask_token = '[MASK]')

In [ ]:
my_tokenizer.padding_side='right'

In [ ]:
my_tokenizer('이것은','다른가',padding='max_length',truncation='longest_first',max_length=max_len)

{'input_ids': [1487, 4, 218, 27608, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
def choose_mask(word,vocab_size):
  if torch.rand(1)<=0.8:
    return '[MASK]'
  elif torch.rand(1) <= 0.5:
    return my_tokenizer.decode(torch.randint(0,vocab_size+7,1))
  return word
  
def masking_sentence(words):
  return [word  if value<=0.85 else "[MASK]" for word,value in zip(words,torch.rand(len(words)))]

In [ ]:
def trans_subset(sen1,sen2,tokenizer):
  is_next = True
  if torch.rand(1)>=0.5:
    sen1,sen2 = sen2,sen1
    is_next = False

  masked_sen1 = " ".join(masking_sentence(sen1.split(" ")))
  masked_sen2 = " ".join(masking_sentence(sen2.split(" ")))
  
  tokened_orig = my_tokenizer(sen1,sen2,padding='max_length',truncation='longest_first',max_length=max_len)
  tokened_mask = my_tokenizer(masked_sen1,sen2,padding='max_length',truncation='longest_first',max_length=max_len)

  return tokened_mask.input_ids,tokened_mask.token_type_ids, tokened_mask.attention_mask, tokened_orig.input_ids,is_next

In [ ]:
if not os.path.exists(os_path+'sentences.txt'):
  if not os.path.exists(drive_path+'sentences.txt'):
    with open(my_path,'r') as f:
      with open(os_path+'sentences.txt','w') as fw:
        is_new=False
        for line in f:
          if line =='\n':
            is_new=False
          else:
            if is_new:
              fw.write(line.rstrip('\n') + ' ')
            else:
              if is_new == False:
                is_new=True
                fw.write('\n')
      shutil.copyfile(os_path+'sentences.txt',drive_path+'sentences.txt')
  else:
    shutil.copyfile(drive_path+'sentences.txt',os_path+'sentences.txt')

In [ ]:
if not (os.path.exists(os_path+'pairs.txt') and os.path.exists(drive_path+'pairs.txt')):
  with open(os_path+'sentences.txt','r') as f:
    with open(os_path+'pairs.txt','w') as fw:
      for strings in f:
          sep_strings = strings.split(". ")
          for str1,str2 in zip(sep_strings[:-2],sep_strings[1:-1]):
            fw.write(str1+'.'+str2+'\n')
    shutil.copyfile(os_path+'pairs.txt',drive_path+'pairs.txt')
else:
  shutil.copyfile(drive_path+'pairs.txt',os_path+'pairs.txt')

In [ ]:
with open(os_path+'pairs.txt','r') as fw:
  my_data =pd.DataFrame([sen for sen in fw])

In [ ]:
my_data = my_data.applymap(lambda x: x.split('.'))

In [ ]:
my_data

In [ ]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.data = my_data.to_numpy()
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    return trans_subset(self.data[idx][0][0]+'.',self.data[idx][0][1].rstrip('\n')+'.',my_tokenizer)

In [ ]:
my_dataset = MyDataset()
my_loader = torch.utils.data.DataLoader(my_dataset)

In [ ]:
my_dataset[0]

([273,
  27599,
  4,
  30,
  27599,
  4,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  1,
  1,
  1,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [273,
  27599,
  4,
  30,
  27599,
  4,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
bert_configure = BertConfig(vocab_size = vocab_size+7)
my_bert = BertModel(bert_configure)